In [1]:
#import modules
import pandas as pd
import json
import matplotlib.pyplot as plt
import numpy as np
import os
import csv
from pandas.api.types import CategoricalDtype
low_memory = False

In [2]:
#read csv data from the CDC for CA
csv_ca_cdc_path = "./Resources/CA_cases_by_county/cdc_ca_output_data.csv"
ca_covid = pd.read_csv(csv_ca_cdc_path, encoding="utf-8")
ca_covid = pd.DataFrame(ca_covid)

C:\Users\Dell\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (6,7) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [3]:
#read in Population Data from the Department of Finance (which the CDC uses)
#as well as partial/full vacciantion numbers
ca_pop_vac_path = './Resources/CA_county_population_and_vaccination_data.csv'
ca_pop_vac = pd.read_csv(ca_pop_vac_path)
ca_popvac_df = pd.DataFrame(ca_pop_vac)

In [4]:
#make a list which counties are not included, though this may not be necessary for the final notebook.
#it is important to mention in the presentation, however.
print(len(ca_covid['res_county'].unique()))

cdc_included_counties = ca_covid['res_county'].unique()
cdc_included_counties = cdc_included_counties.tolist()

counties_list = ['Alameda', 'Alpine', 'Amador' ,'Butte' ,'Calaveras' ,'Colusa' ,'Contra Costa' ,'Del Norte'
                 ,'El Dorado' ,'Fresno' ,'Glenn' ,'Humboldt' ,'Imperial' ,'Inyo' ,'Kern' ,'Kings' ,'Lake' ,'Lassen',
                 'Los Angeles' ,'Madera' ,'Marin' ,'Mariposa' ,'Mendocino' ,'Merced' ,'Modoc' ,'Mono' ,'Monterey' ,'Napa', 
                 'Nevada' ,'Orange' ,'Placer' ,'Plumas' ,'Riverside' ,'Sacramento' ,'San Benito' ,'San Bernardino' ,'San Diego' ,
                 'San Francisco' ,'San Joaquin' ,'San Luis Obispo' ,'San Mateo' ,'Santa Barbara' ,'Santa Clara' ,'Santa Cruz' ,
                 'Shasta' ,'Sierra' ,'Siskiyou' ,'Solano' ,'Sonoma' ,'Stanislaus' ,'Sutter' ,'Tehama' ,'Trinity' ,'Tulare' ,'Tuolumne' ,
                 'Ventura' ,'Yolo' ,'Yuba']

small_county = []
#print(len(counties_list))
for county in counties_list:
    if county.upper() not in cdc_included_counties:
        small_county.append(county)
#print(small_county)

51


In [5]:
#extract only the data for dates in 2021
ca_covid_2021 = ca_covid.loc[(ca_covid['case_month'] == '2021-01') | (ca_covid['case_month'] == '2021-02') | 
                              (ca_covid['case_month'] == '2021-03') | (ca_covid['case_month'] == '2021-04')  
                            | (ca_covid['case_month'] == '2021-05') | (ca_covid['case_month'] == '2021-06') 
                            | (ca_covid['case_month'] == '2021-07') ]                           

ca_covid_2021.head()

,case_month,res_state,res_county,age_group,current_status,sex,race,ethnicity
2,2021-01,CA,SANTA CLARA,0 - 17 years,Laboratory-confirmed case,Female,White,Non-Hispanic/Latino
7,2021-02,CA,SANTA BARBARA,0 - 17 years,Laboratory-confirmed case,Female,White,Non-Hispanic/Latino
9,2021-01,CA,VENTURA,18 to 49 years,Laboratory-confirmed case,Female,American Indian/Alaska Native,Hispanic/Latino
13,2021-03,CA,NEVADA,0 - 17 years,Laboratory-confirmed case,Female,White,Non-Hispanic/Latino
18,2021-02,CA,SOLANO,18 to 49 years,Laboratory-confirmed case,Female,White,Non-Hispanic/Latino


In [6]:
#get the value counts for demographics and case status

#print(ca_covid.head(70))
age_counts = ca_covid_2021['age_group'].value_counts()
print(age_counts)
case_status_counts = ca_covid_2021['current_status'].value_counts()
#print(case_status_counts)
sex_counts = ca_covid_2021['sex'].value_counts()
#print(sex_counts)
race_counts = ca_covid_2021['race'].value_counts()
#print(race_counts)
ethnicity_counts = ca_covid_2021['ethnicity'].value_counts
#print(ethnicity_counts)


#write out the modified dataframe
ca_covid_2021 = ca_covid_2021.drop(columns=['res_state'])
ca_covid_2021.head()
ca_covid_2021.to_csv('./Resources/CA_cases_by_county/ca_2021_case_data.csv',
                    index=False,header=True)

18 to 49 years    755943
50 to 64 years    248834
0 - 17 years      201041
65+ years         136457
Missing              548
Name: age_group, dtype: int64


In [7]:
#convert the case_month to month names so that it can merge with Tikaram
#use DatetimeIndex so that computer does not time out in for loop
ca_covid_2021['month'] = pd.DatetimeIndex(ca_covid_2021['case_month']).month_name()

In [8]:
# ordering months to be sorted by position in the calendar rather than by alphabetical order
#for graphing purposes. Code derived from Tikaram's notebook, and Andrew Morrision showed us how
#to use the CategoricalDtype
month = ca_covid_2021["month"]

month_order = CategoricalDtype(['December', 'January', 'February', 'March', 'April', \
                                'May', 'June', 'July', 'August'], ordered=True)

ca_covid_2021['month'] = ca_covid_2021['month'].astype(month_order)


In [9]:
# make smaller dataframes for each demographic with county, case month, current_status,
# and demographic in the individual dataframes

age_df = ca_covid_2021[['age_group','month', 'res_county', 'current_status']]
age_df.head(3)

,age_group,month,res_county,current_status
2,0 - 17 years,January,SANTA CLARA,Laboratory-confirmed case
7,0 - 17 years,February,SANTA BARBARA,Laboratory-confirmed case
9,18 to 49 years,January,VENTURA,Laboratory-confirmed case


In [10]:
#bar graph goes here
#x_axis = [1,2,3]
#y_axis = age_df['age_group'].value_counts()
#plt.bar(x_axis,y_axis)

In [11]:
sex_df = ca_covid_2021[['sex','month', 'res_county', 'current_status']]
sex_df.head(3)


,sex,month,res_county,current_status
2,Female,January,SANTA CLARA,Laboratory-confirmed case
7,Female,February,SANTA BARBARA,Laboratory-confirmed case
9,Female,January,VENTURA,Laboratory-confirmed case


In [12]:
race_df = ca_covid_2021[['race','month', 'res_county', 'current_status']]
race_df.head(3)

,race,month,res_county,current_status
2,White,January,SANTA CLARA,Laboratory-confirmed case
7,White,February,SANTA BARBARA,Laboratory-confirmed case
9,American Indian/Alaska Native,January,VENTURA,Laboratory-confirmed case


In [13]:
ethnicity_df = ca_covid_2021[['ethnicity','month', 'res_county', 'current_status']]
ethnicity_df.head(3)
ethnicity_df['ethnicity'].value_counts()

Non-Hispanic/Latino    434808
Unknown                215134
Hispanic/Latino        186220
Name: ethnicity, dtype: int64